# Experiment writing to a zarr store on the JASMIN s3 object store.

* Working with `s3fs`, not with `fsspec`
* Cannot write large files (3D zoom level 10) - run out of mem.

In [16]:
from pathlib import Path

import fsspec
import s3fs
import xarray as xr
import zarr

In [8]:
dsT = xr.open_dataset('/gws/nopw/j04/hrcm/mmuetz/DYAMOND3_example_data/healpix/air_temperature/hpz0/glm.n2560_RAL3p3.apverd_20200120T00.air_temperature.hpz0.nc')


/home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)
/tmp/ipykernel_1909/333485658.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dsT = xr.open_dataset('/gws/nopw/j04/hrcm/mmuetz/DYAMOND3_example_data/healpix/air_temperature/hpz0/glm.n2560_RAL3p3.apverd_20200120T00.air_temperature.hpz0.nc')


In [9]:
dsT

<xarray.Dataset> Size: 12kB
Dimensions:                  (time: 5, pressure: 25, cell: 12)
Coordinates:
  * time                     (time) datetime64[ns] 40B 2020-01-20 ... 2020-01...
  * pressure                 (pressure) float32 100B 1.0 5.0 ... 975.0 1e+03
    forecast_period          (time) timedelta64[ns] 40B ...
    forecast_reference_time  datetime64[ns] 8B ...
  * cell                     (cell) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
Data variables:
    air_temperature          (time, pressure, cell) float64 12kB ...
Attributes:
    standard_name:    air_temperature
    units:            K
    um_stash_source:  m01s16i203
    source:           Data from Met Office Unified Model
    um_version:       13.5
    grid_mapping:     healpix_nested
    healpix_zoom:     0

In [2]:
s3cfg = dict([l.split(' = ') for l in Path('/home/users/mmuetz/.s3cfg').read_text().split('\n') if l])

In [3]:
# Internal access allowed in notebooks - no https and s3-ext.
jasmin_s3 = s3fs.S3FileSystem(
    anon=False, secret=s3cfg['secret_key'],
    key=s3cfg['access_key'],
    client_kwargs={'endpoint_url': 'http://hackathon-o.s3.jc.rl.ac.uk'}
)
jasmin_s3.ls('data')

In [12]:
store= s3fs.S3Map(root='s3://data/hpz0/glm.n2560_RAL3p3.apverd_20200120T00.air_temperature.hpz0.zarr', s3=jasmin_s3, check=False)
task = dsT.to_zarr(store=store)

In [14]:
dsT2 = xr.open_zarr(store=store)

/tmp/ipykernel_1909/336796369.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dsT2 = xr.open_zarr(store=store)


In [15]:
dsT2

<xarray.Dataset> Size: 12kB
Dimensions:                  (time: 5, pressure: 25, cell: 12)
Coordinates:
  * cell                     (cell) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
    forecast_period          (time) timedelta64[ns] 40B dask.array<chunksize=(5,), meta=np.ndarray>
    forecast_reference_time  datetime64[ns] 8B ...
  * pressure                 (pressure) float32 100B 1.0 5.0 ... 975.0 1e+03
  * time                     (time) datetime64[ns] 40B 2020-01-20 ... 2020-01...
Data variables:
    air_temperature          (time, pressure, cell) float64 12kB dask.array<chunksize=(5, 25, 12), meta=np.ndarray>
Attributes:
    grid_mapping:     healpix_nested
    healpix_zoom:     0
    source:           Data from Met Office Unified Model
    standard_name:    air_temperature
    um_stash_source:  m01s16i203
    um_version:       13.5
    units:            K

In [17]:
dsT2.air_temperature.sum().compute()

<xarray.DataArray 'air_temperature' ()> Size: 8B
array(376112.32569437)
Coordinates:
    forecast_reference_time  datetime64[ns] 8B 2020-01-20

In [25]:
# Try writing a much larger dataset (zoom level 10).
dsT = xr.open_dataset('/gws/nopw/j04/hrcm/mmuetz/DYAMOND3_example_data/healpix/air_temperature/hpz10/glm.n2560_RAL3p3.apverd_20200120T00.air_temperature.hpz10.nc')


/tmp/ipykernel_1909/1905326407.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dsT = xr.open_dataset('/gws/nopw/j04/hrcm/mmuetz/DYAMOND3_example_data/healpix/air_temperature/hpz10/glm.n2560_RAL3p3.apverd_20200120T00.air_temperature.hpz10.nc')


In [ ]:
# Runs out of mem with 24G, even though compute=False.
store= s3fs.S3Map(root='s3://data/hpz10/glm.n2560_RAL3p3.apverd_20200120T00.air_temperature.hpz10.zarr', s3=jasmin_s3, check=False)
task = dsT.to_zarr(store=store, compute=False)

## experiment with `fsspec`

* Basic access to s3 store works...
* But, I cannot get this working with `xarray` reading from a store that I've set up using `fsspec`

In [5]:
fs = fsspec.filesystem(
    's3',
    key=s3cfg['access_key'],
    secret=s3cfg['secret_key'],
    client_kwargs={'endpoint_url': 'http://hackathon-o.s3.jc.rl.ac.uk'},  # Works locally withing JASMIN/Notebook service.
)

In [6]:
fs.ls('data')

['data/20200101T0000Z_ph2496.pp', 'data/zarr_example.zarr']

In [19]:
s3_store = fsspec.get_mapper(
    "s3://data/hpz0/glm.n2560_RAL3p3.apverd_20200120T00.air_temperature.hpz0.zarr",
    storage_options={
        'key': s3cfg['access_key'],
        'secret': s3cfg['secret_key'],        
        "client_kwargs": {"endpoint_url": "http://hackathon-o.s3.jc.rl.ac.uk"}  # Adjust for your S3-compatible storage
    }
)

In [23]:
s3_store = fsspec.get_mapper("s3://data/hpz0/glm.n2560_RAL3p3.apverd_20200120T00.air_temperature.hpz0.zarr", fs=fs)

In [24]:
dsT3 = xr.open_zarr(s3_store)

TypeError: AioSession.__init__() got an unexpected keyword argument 'fs'